In [1]:
import pandas as pd
# from crim_intervals.funcs import *
%cd ../intervals
from main_objs import *
from main import *

from ipywidgets import interact
import re

/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


In [2]:
def map_colours(x, rest_colour="#fbb4ae", empty_colour="white", remainder_colour="#ccebc5"):
    if x == "Rest":
        return f"background: {rest_colour}"
    elif pd.isnull(x) or (not x) or (x == "-"):
        return f"background: {empty_colour}"
    else:
        return f"background: {remainder_colour}"
    
    
def add_offset(s, lyric_line=3):
    for n in s.recurse().getElementsByClass('Note'):
        n.addLyric(int(n.getOffsetBySite(s.flat.notes)), lyric_line)

# Select Piece

In [3]:
piece = 'https://crimproject.org/mei/CRIM_Mass_0002_1.mei'
corpus = CorpusBase([piece])

Successfully imported.


In [4]:
model = corpus.scores[0]

In [5]:
model.score.metadata.title

'Missa Vidi speciosam - Kyrie'

In [6]:
add_offset(model.score)
model.score.measures(1, 15).show()

SubConverterException: Cannot find a path to the 'mscore' file at /Applications/MuseScore 3.app/Contents/MacOS/mscore -- download MuseScore

# Music as a table

In [7]:
model.getNoteRest().head(10).fillna("-").T.style.applymap(map_colours)

,0.0,4.0,8.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0
[Superius],G4,D5,-,C5,D5,F5,E5,D5,C5,A4
[Contratenor],Rest,-,Rest,-,D4,-,G4,-,-,F4
I. Tenor,Rest,-,Rest,-,-,-,Rest,-,-,-
[II. Tenor],Rest,-,Rest,-,-,-,Rest,-,-,-
[Bassus],Rest,-,Rest,-,-,-,Rest,-,-,-


# Melodic patterns

In [8]:
model_melodic = model.getMelodic(kind="d", unit=2)
# ave_melodic

In [9]:
ngrams = model.getNgrams(df=model.getMelodic(kind="d", compound=True), n=5)
ngrams

,[Superius],[Contratenor],I. Tenor,[II. Tenor],[Bassus]
4.0,"(5, -2, 2, 3, -2)",NaN,NaN,NaN,NaN
10.0,"(-2, 2, 3, -2, -2)",NaN,NaN,NaN,NaN
12.0,"(2, 3, -2, -2, -2)",NaN,NaN,NaN,NaN
14.0,"(3, -2, -2, -2, -3)",NaN,NaN,NaN,NaN
16.0,"(-2, -2, -2, -3, 2)","(4, -2, 2, 3, -2)",NaN,NaN,NaN
...,...,...,...,...,...
524.0,NaN,"(-5, 3, 2, -2, 1)",NaN,NaN,NaN
525.0,NaN,NaN,"(-2, -2, -2, -2, -2)",NaN,NaN
526.0,"(-2, -2, -2, 2, 2)",NaN,"(-2, -2, -2, -2, -2)",NaN,NaN
536.0,NaN,NaN,NaN,"(2, 3, -2, -2, -2)",NaN


## Searching with various parameters
* Note that if 'units' are no specified, then the results are simply the melodic vectors, regardless of duration.
* If we specify "units" then the results are incremental by that unit
* The results will be different in each case
* Sample search: 4, 1, 2, 2 or chromatic 5, 0, 2, 2, -4

In [10]:
# Ngrams without time_units
@interact(search_pattern="", kind=["d", "c"])
def get_ngrams(search_pattern, kind):
    ngrams = model.getNgrams(df=model.getMelodic(kind=kind), n=5)
    mask = ngrams.apply(lambda x: x.astype(str).str.contains(search_pattern).any(), axis=1)
    filtered_ngrams = ngrams[mask].copy()
    return filtered_ngrams.fillna("-").style.applymap(lambda x: "background: #ccebc5" if search_pattern in x else "")

interactive(children=(Text(value='', description='search_pattern'), Dropdown(description='kind', options=('d',…

In [11]:
# NGrams with time_units

@interact(search_pattern="", kind=["d", "c"], time_unit=[1, 2, 4, 8], length=[3, 4, 5, 6, 7])
def get_ngrams(search_pattern, kind, time_unit, length):
    ngrams = model.getNgrams(df=model.getMelodic(kind=kind), n=length, unit=time_unit)
    mask = ngrams.apply(lambda x: x.astype(str).str.contains(search_pattern).any(), axis=1)
    filtered_ngrams = ngrams[mask].copy()
    return filtered_ngrams.fillna("-").style.applymap(lambda x: "background: #ccebc5" if search_pattern in x else "")

interactive(children=(Text(value='', description='search_pattern'), Dropdown(description='kind', options=('d',…

# Count of melodic intervals

In [12]:
model.getMelodic(directed=True).apply(pd.Series.value_counts).fillna(0).astype(int).style.bar()

,[Superius],[Contratenor],I. Tenor,[II. Tenor],[Bassus]
-M2,47,35,41,40,25
-M3,5,4,4,2,4
-P4,1,1,0,3,3
-P5,0,3,3,1,3
-P8,0,1,0,0,0
-m2,25,21,14,17,13
-m3,8,9,10,9,4
M2,28,29,29,29,24
M3,3,4,1,2,0
P1,5,2,1,1,1


# Harmonic intervals

Example search pattern: 12, 10, 8, 8|5, 3, 1, 1$

Authentic cadence:  7, 6, 8

In [13]:
# note that time_unit does not yet work with getHarmonic
@interact(search_pattern="", compound=[True, False], kind=["d", "c"], time_unit=[1, 2, 4, 8], length=[3, 4, 5, 6, 7])
def get_harmonic_ngrams(search_pattern, compound, kind, time_unit, length):    
    model_harmonic = model.getHarmonic(kind=kind, compound=compound)
    ngrams = model.getNgrams(df=model_harmonic, n=length, cell_type="str", unit=time_unit)
    filtered_ngrams = ngrams[ngrams.apply(lambda x: x.astype(str).str.contains(search_pattern).any(), axis=1)].copy()
    return filtered_ngrams.fillna("-").style.applymap(lambda x: "background: #ccebc5" if re.match(search_pattern, x) else "")


interactive(children=(Text(value='', description='search_pattern'), Dropdown(description='compound', options=(…

In [14]:
har = model.getHarmonic(kind="d")
regHar = model.regularize(df=har, unit=1)
regHar

,[Bassus]_[II. Tenor],[Bassus]_I. Tenor,[Bassus]_[Contratenor],[Bassus]_[Superius],[II. Tenor]_I. Tenor,[II. Tenor]_[Contratenor],[II. Tenor]_[Superius],I. Tenor_[Contratenor],I. Tenor_[Superius],[Contratenor]_[Superius]
0.0,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest
1.0,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest
2.0,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest
3.0,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest
4.0,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest,Rest
...,...,...,...,...,...,...,...,...,...,...
555.0,3,1,5,8,-3,3,6,5,8,4
556.0,3,1,5,8,-3,3,6,5,8,4
557.0,3,1,5,8,-3,3,6,5,8,4
558.0,3,1,5,8,-3,3,6,5,8,4


In [15]:
mel = model.getNgrams(df=model.getMelodic(kind="d"), n=4)
# regMel = ave_maria.regularize(df=mel, unit=4)
# regMel.stack().to_frame().reset_index(level=1)
b = mel.stack().to_frame().reset_index(level=1)
b.groupby("level_1")

## Return offsets as list
### For use with filtered DFs from classifier, etc.

In [16]:
df = get_harmonic_ngrams("7, 6, 8", True, "d", 1, 3)
a = df.index.tolist()
a
for i in a:
    b = i/8 - 1
    print(b)

14.0
17.0
17.5
40.5
55.75


# Modules:

* Authentic/Phrygian cadence with suspension in diatonic:  7_Held, 6_-2, 8
* Authentic/Phrygian cadence with suspension in chromatic: 11_Held, 9_-2, 12
* Authentic/Phrygian cadence with suspension in zero-based:  6_Held, 5_-2, 7



In [17]:
@interact(search_pattern="", compound=[True, False], time_unit=[4, 2, 1, 8], length=[3, 4, 5, 6])
def get_modules(search_pattern, compound, time_unit, length):
    
    model_har = model.getHarmonic(kind="d", compound=compound)
    ngrams = model.getNgrams(df=model_har, n=length, how='modules', exclude=['Rest'], cell_type="str", unit=time_unit)
    filtered_ngrams = ngrams[ngrams.apply(lambda x: x.astype(str).str.contains(search_pattern).any(), axis=1)].copy()
    return filtered_ngrams.fillna("-").style.applymap(lambda x: "background: #ccebc5" if re.match(search_pattern, x) else "")

interactive(children=(Text(value='', description='search_pattern'), Dropdown(description='compound', options=(…

In [18]:
model_har = model.getHarmonic(kind="d", compound=True)
ngrams = model.getNgrams(df=model_har, n=3, how='modules', exclude=['Rest'], cell_type="str")
# filtered_ngrams = ngrams[ngrams.apply(lambda x: x.astype(str).str.contains('7').any(), axis=1)].copy()
# filtered_ngrams.stack().value_counts().to_frame()
ngrams.stack().value_counts().to_frame()

,0
"3_Held, 2_Held, 3",8
"6_-2, 7_-2, 8",8
"5_-2, 6_-2, 7",7
"3_Held, 2_Held, 1",7
"4_Held, 3_Held, 2",7
...,...
"6_-2, 5_-3, 4",1
"3_-3, 6_-3, 8",1
"3_-8, 10_3, 8",1
"6_-2, 8_-2, 9",1


In [ ]:
ave_maria.score.measures(1, 4).show()